# 网络数据处理与分析项目

这个 Notebook 集成了网络数据处理与分析的多个执行步骤，包括数据收集、预处理、分析和可视化。

## 目录
1. [环境设置与导入库](#1-环境设置与导入库)
2. [数据收集](#2-数据收集)
3. [数据预处理](#3-数据预处理)
4. [网络分析](#4-网络分析)
5. [数据可视化](#5-数据可视化)
6. [结果导出](#6-结果导出)

## 1. 环境设置与导入库

In [1]:
# 导入必要的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import requests
from bs4 import BeautifulSoup
import ipaddress
from pathlib import Path
import warnings

# 设置显示选项
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']  # 支持中文显示
plt.rcParams['axes.unicode_minus'] = False  # 正常显示负号
sns.set_style("whitegrid")
warnings.filterwarnings('ignore')

# 设置数据目录
DATA_DIR = Path('data')
DATA_DIR.mkdir(exist_ok=True)

print("✅ 环境设置完成")
print(f"📁 数据目录: {DATA_DIR.absolute()}")

✅ 环境设置完成
📁 数据目录: /Users/zhangxudong/Gits/network/data


## 2. 数据收集

这一步骤包括从各种来源收集网络数据。

In [ ]:
# 示例：创建模拟网络数据
def generate_sample_network_data():
    """生成示例网络数据"""
    np.random.seed(42)
    
    # 生成节点数据
    nodes = []
    for i in range(50):
        node = {
            'id': i,
            'name': f'Node_{i}',
            'type': np.random.choice(['server', 'client', 'router'], p=[0.2, 0.6, 0.2]),
            'ip': str(ipaddress.IPv4Address('192.168.1.1') + i),
            'status': np.random.choice(['active', 'inactive'], p=[0.8, 0.2])
        }
        nodes.append(node)
    
    # 生成边数据（连接关系）
    edges = []
    for i in range(80):
        source = np.random.randint(0, 50)
        target = np.random.randint(0, 50)
        if source != target:
            edge = {
                'source': source,
                'target': target,
                'weight': np.random.uniform(0.1, 1.0),
                'protocol': np.random.choice(['TCP', 'UDP', 'HTTP'], p=[0.5, 0.3, 0.2]),
                'bandwidth': np.random.uniform(10, 1000)  # Mbps
            }
            edges.append(edge)
    
    return pd.DataFrame(nodes), pd.DataFrame(edges)

# 生成示例数据
nodes_df, edges_df = generate_sample_network_data()

print("📊 数据收集完成")
print(f"节点数量: {len(nodes_df)}")
print(f"连接数量: {len(edges_df)}")

# 显示数据样本
print("\n节点数据样本:")
display(nodes_df.head())

print("\n连接数据样本:")
display(edges_df.head())

## 3. 数据预处理

清理和准备数据用于分析。

In [ ]:
# 数据清理和预处理
def preprocess_network_data(nodes_df, edges_df):
    """预处理网络数据"""
    # 清理节点数据
    nodes_clean = nodes_df.copy()
    nodes_clean = nodes_clean.dropna()  # 删除空值
    
    # 清理边数据
    edges_clean = edges_df.copy()
    edges_clean = edges_clean.dropna()
    
    # 删除重复的边
    edges_clean = edges_clean.drop_duplicates(subset=['source', 'target'])
    
    # 确保边的节点存在于节点列表中
    valid_nodes = set(nodes_clean['id'])
    edges_clean = edges_clean[
        (edges_clean['source'].isin(valid_nodes)) & 
        (edges_clean['target'].isin(valid_nodes))
    ]
    
    return nodes_clean, edges_clean

# 预处理数据
nodes_clean, edges_clean = preprocess_network_data(nodes_df, edges_df)

print("🧹 数据预处理完成")
print(f"清理后节点数量: {len(nodes_clean)}")
print(f"清理后连接数量: {len(edges_clean)}")

# 数据统计
print("\n📈 数据统计:")
print("节点类型分布:")
print(nodes_clean['type'].value_counts())

print("\n协议分布:")
print(edges_clean['protocol'].value_counts())

## 4. 网络分析

使用 NetworkX 进行网络拓扑分析。

In [ ]:
# 创建网络图
def create_network_graph(nodes_df, edges_df):
    """创建 NetworkX 图对象"""
    G = nx.Graph()
    
    # 添加节点
    for _, node in nodes_df.iterrows():
        G.add_node(node['id'], **node.to_dict())
    
    # 添加边
    for _, edge in edges_df.iterrows():
        G.add_edge(edge['source'], edge['target'], **edge.to_dict())
    
    return G

# 创建网络图
G = create_network_graph(nodes_clean, edges_clean)

print("🕸️ 网络图创建完成")
print(f"节点数: {G.number_of_nodes()}")
print(f"边数: {G.number_of_edges()}")
print(f"密度: {nx.density(G):.4f}")

# 计算网络指标
print("\n📊 网络分析指标:")

# 度中心性
degree_centrality = nx.degree_centrality(G)
print(f"平均度中心性: {np.mean(list(degree_centrality.values())):.4f}")

# 聚类系数
clustering = nx.clustering(G)
avg_clustering = np.mean(list(clustering.values()))
print(f"平均聚类系数: {avg_clustering:.4f}")

# 连通分量
connected_components = list(nx.connected_components(G))
print(f"连通分量数: {len(connected_components)}")
print(f"最大连通分量大小: {len(max(connected_components, key=len))}")

# 最重要的节点（按度中心性）
top_nodes = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:5]
print("\n🔝 度中心性最高的5个节点:")
for node_id, centrality in top_nodes:
    node_info = nodes_clean[nodes_clean['id'] == node_id].iloc[0]
    print(f"  {node_info['name']} ({node_info['type']}): {centrality:.4f}")

## 5. 数据可视化

创建各种图表来可视化网络数据和分析结果。

In [ ]:
# 设置图形大小
plt.figure(figsize=(20, 15))

# 1. 网络拓扑图
plt.subplot(2, 3, 1)
pos = nx.spring_layout(G, k=1, iterations=50)

# 根据节点类型设置颜色
node_colors = []
for node in G.nodes():
    node_type = G.nodes[node]['type']
    if node_type == 'server':
        node_colors.append('red')
    elif node_type == 'client':
        node_colors.append('blue')
    else:  # router
        node_colors.append('green')

nx.draw(G, pos, node_color=node_colors, node_size=100, 
        with_labels=False, edge_color='gray', alpha=0.7)
plt.title('网络拓扑图')

# 2. 节点类型分布
plt.subplot(2, 3, 2)
type_counts = nodes_clean['type'].value_counts()
plt.pie(type_counts.values, labels=type_counts.index, autopct='%1.1f%%')
plt.title('节点类型分布')

# 3. 度分布
plt.subplot(2, 3, 3)
degrees = [G.degree(n) for n in G.nodes()]
plt.hist(degrees, bins=20, alpha=0.7, edgecolor='black')
plt.xlabel('度')
plt.ylabel('频次')
plt.title('度分布')

# 4. 协议分布
plt.subplot(2, 3, 4)
protocol_counts = edges_clean['protocol'].value_counts()
plt.bar(protocol_counts.index, protocol_counts.values)
plt.xlabel('协议类型')
plt.ylabel('连接数')
plt.title('协议分布')

# 5. 带宽分布
plt.subplot(2, 3, 5)
plt.hist(edges_clean['bandwidth'], bins=20, alpha=0.7, edgecolor='black')
plt.xlabel('带宽 (Mbps)')
plt.ylabel('频次')
plt.title('带宽分布')

# 6. 度中心性分布
plt.subplot(2, 3, 6)
centrality_values = list(degree_centrality.values())
plt.hist(centrality_values, bins=20, alpha=0.7, edgecolor='black')
plt.xlabel('度中心性')
plt.ylabel('频次')
plt.title('度中心性分布')

plt.tight_layout()
plt.show()

print("📈 可视化完成")

In [ ]:
# 创建更详细的网络可视化
plt.figure(figsize=(15, 10))

# 使用度中心性调整节点大小
node_sizes = [degree_centrality[node] * 1000 + 50 for node in G.nodes()]

# 绘制网络图
pos = nx.spring_layout(G, k=2, iterations=100)
nx.draw_networkx_nodes(G, pos, node_color=node_colors, 
                      node_size=node_sizes, alpha=0.8)
nx.draw_networkx_edges(G, pos, alpha=0.3, edge_color='gray')

# 添加标签（只显示重要节点）
important_nodes = [node for node, centrality in degree_centrality.items() 
                  if centrality > np.percentile(list(degree_centrality.values()), 80)]
labels = {node: f"N{node}" for node in important_nodes}
nx.draw_networkx_labels(G, pos, labels, font_size=8)

plt.title('网络拓扑图（节点大小表示度中心性）', fontsize=16)
plt.axis('off')

# 添加图例
legend_elements = [
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='red', 
               markersize=10, label='服务器'),
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', 
               markersize=10, label='客户端'),
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='green', 
               markersize=10, label='路由器')
]
plt.legend(handles=legend_elements, loc='upper right')

plt.show()

## 6. 结果导出

保存分析结果和可视化图表。

In [ ]:
# 创建结果目录
RESULTS_DIR = Path('results')
RESULTS_DIR.mkdir(exist_ok=True)

# 保存数据
nodes_clean.to_csv(RESULTS_DIR / 'nodes_processed.csv', index=False)
edges_clean.to_csv(RESULTS_DIR / 'edges_processed.csv', index=False)

# 保存网络指标
network_metrics = {
    'total_nodes': G.number_of_nodes(),
    'total_edges': G.number_of_edges(),
    'density': nx.density(G),
    'average_degree_centrality': np.mean(list(degree_centrality.values())),
    'average_clustering_coefficient': avg_clustering,
    'connected_components': len(connected_components),
    'largest_component_size': len(max(connected_components, key=len))
}

metrics_df = pd.DataFrame([network_metrics])
metrics_df.to_csv(RESULTS_DIR / 'network_metrics.csv', index=False)

# 保存中心性指标
centrality_df = pd.DataFrame([
    {'node_id': node, 'degree_centrality': centrality}
    for node, centrality in degree_centrality.items()
])
centrality_df = centrality_df.merge(nodes_clean[['id', 'name', 'type']], 
                                   left_on='node_id', right_on='id')
centrality_df.to_csv(RESULTS_DIR / 'node_centrality.csv', index=False)

print("💾 结果导出完成")
print(f"📁 结果保存在: {RESULTS_DIR.absolute()}")
print("\n导出的文件:")
for file in RESULTS_DIR.glob('*.csv'):
    print(f"  - {file.name}")

# 显示最终总结
print("\n📋 分析总结:")
print(f"  • 处理了 {len(nodes_clean)} 个节点和 {len(edges_clean)} 条连接")
print(f"  • 网络密度: {nx.density(G):.4f}")
print(f"  • 平均聚类系数: {avg_clustering:.4f}")
print(f"  • 最重要的节点: {nodes_clean[nodes_clean['id'] == top_nodes[0][0]]['name'].iloc[0]}")

## 总结

这个 Notebook 提供了一个完整的网络数据处理与分析流程：

1. **环境设置**: 导入必要的库并配置环境
2. **数据收集**: 生成或加载网络数据
3. **数据预处理**: 清理和准备数据
4. **网络分析**: 计算各种网络指标
5. **数据可视化**: 创建图表展示分析结果
6. **结果导出**: 保存处理后的数据和分析结果

您可以根据具体需求修改和扩展这些步骤。